# TP1 - IBD - Bruno Varella Peixoto
## Matrícula: 2014004158

In [0]:
import io
import sqlite3
import pandas as pd

###Download dos dados

In [0]:
!wget --quiet -P /tmp/ https://raw.githubusercontent.com/BVarella/TP1-IBD/master/despesas.sql

### Carregando os dados

In [0]:
conn = sqlite3.connect('/tmp/despesas.db')
cursor = conn.cursor()

In [4]:
f = io.open('/tmp/despesas.sql', 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

##Exercícios SQL

Como minha matrícula é 2014**004158**, devo fazer as consultas de número: 1, 2, 5, 3, 6, 9

### 1. Liste todos os códigos e nomes de subfunções da função que possui o maior número de subfunções.

In [5]:
df = pd.read_sql_query("SELECT COD_SUBFUNCAO, NOME_SUBFUNCAO FROM (SELECT * FROM FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO NATURAL JOIN SUBFUNCAO) WHERE COD_FUNCAO IN (SELECT COD_FUNCAO FROM (SELECT COUNT(COD_FUNCAO) AS NUM_SUBFUNCOES, COD_FUNCAO FROM DETALHAMENTO_FUNCAO GROUP BY(COD_FUNCAO) ORDER BY NUM_SUBFUNCOES DESC LIMIT 1))", conn)
df

,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,125,Normatização e fiscalização
1,126,Tecnologia da informação
2,131,Comunicação social
3,244,Assistência comunitária
4,306,Alimentação e nutrição
5,365,Educação infantil
6,511,Saneamento básico rural


### 2. Liste o valor total de despesas públicas por função (nome) e subfunção (nome), apenas nos casos em que esse valor total excedeu 200 mil reais.

In [6]:
df = pd.read_sql_query("SELECT NOME_FUNCAO, NOME_SUBFUNCAO, VALOR_TOTAL FROM (SELECT NOME_FUNCAO, NOME_SUBFUNCAO, SUM(VALOR_DESPESA) AS VALOR_TOTAL FROM (((FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO) NATURAL JOIN SUBFUNCAO) NATURAL JOIN DESPESA_PUBLICA) GROUP BY COD_FUNCAO, COD_SUBFUNCAO) WHERE VALOR_TOTAL > 200000", conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO,VALOR_TOTAL
0,Assistência social,Assistência comunitária,227883.77
1,Educação,Educação básica,334262.06


### 3. Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [7]:
df = pd.read_sql_query("SELECT NOME_ORGAO, SUM(VALOR_DESPESA) AS VALOR_TOTAL FROM (DESPESA_PUBLICA NATURAL JOIN ORGAO) WHERE ANO_DESPESA = 2018 GROUP BY COD_ORGAO, ANO_DESPESA ORDER BY VALOR_TOTAL DESC", conn)
df

,NOME_ORGAO,VALOR_TOTAL
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,Presidência da República,0.00
6,"Ministério da Ciência, Tecnologia, Inovações",0.00
7,Ministério da Fazenda,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


### 5. Liste o código e nome dos programas de governo que possuiram os 3 maiores valores de despesas totais na função de educação durante o ano de 2018.

In [8]:
df = pd.read_sql_query("SELECT COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO FROM (SELECT NOME_FUNCAO, COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO, SUM(VALOR_DESPESA) AS VALOR_TOTAL FROM (((FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO) NATURAL JOIN DESPESA_PUBLICA) NATURAL JOIN PROGRAMA_GOVERNO) WHERE NOME_FUNCAO = 'Educação' GROUP BY COD_PROGRAMA_GOVERNO ORDER BY VALOR_TOTAL DESC LIMIT 3)", conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,11,ENEM
1,14,PRONATEC
2,35,MAIS MEDICOS


### 6. Liste todos os nomes de funções e subfunções das despesas públicas do terceiro trimestre de 2018 e seus respectivos valores totais, apenas para o programa MAIS MEDICOS.


In [9]:
df = pd.read_sql_query("SELECT NOME_FUNCAO, NOME_SUBFUNCAO, SUM(VALOR_DESPESA) AS VALOR_TOTAL FROM (SELECT * FROM ((((FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO) NATURAL JOIN SUBFUNCAO) NATURAL JOIN DESPESA_PUBLICA) NATURAL JOIN PROGRAMA_GOVERNO) WHERE NOME_PROGRAMA_GOVERNO = 'MAIS MEDICOS' AND (MES_DESPESA = 7 OR MES_DESPESA = 8 OR MES_DESPESA = 9)) GROUP BY COD_FUNCAO, COD_SUBFUNCAO", conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO,VALOR_TOTAL
0,Defesa nacional,Administração geral,0
1,Saúde,Atenção básica,11000
2,Educação,Assistência hospitalar e ambulatorial,0
3,Educação,Ensino superior,2400


### 9. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior número de órgãos subordinados.

In [10]:
df = pd.read_sql_query("SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA FROM DESPESA_PUBLICA WHERE COD_ORGAO IN(SELECT COD_ORGAO FROM ORGAO ORDER BY QTD_ORGAOS_SUBORDINADOS DESC LIMIT 1)", conn)
df

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,196,10,2018,Reserva de Contingência,0.0
1,197,1,2018,Reserva de Contingência,0.0
2,198,2,2018,Reserva de Contingência,0.0
3,199,3,2018,Reserva de Contingência,0.0
4,200,4,2018,Reserva de Contingência,0.0
5,201,5,2018,Reserva de Contingência,0.0
6,202,6,2018,Reserva de Contingência,0.0
7,203,7,2018,Reserva de Contingência,0.0
8,204,9,2018,Reserva de Contingência,0.0
9,205,10,2018,Reserva de Contingência,0.0


##Exercícios Álgebra Relacional

### 12 - 

In [11]:
df = pd.read_sql_query("SELECT NOME_FUNCAO, NOME_SUBFUNCAO FROM ((FUNCAO NATURAL JOIN DETALHAMENTO_FUNCAO) NATURAL JOIN SUBFUNCAO)", conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


### 13 - 

In [12]:
df = pd.read_sql_query("SELECT MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO FROM (PROGRAMA_GOVERNO NATURAL JOIN DESPESA_PUBLICA) WHERE MODALIDADE_DESPESA = 'Reserva de Contingência'", conn)
df

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
5,Reserva de Contingência,0.0,BRASIL SEM MISERIA
6,Reserva de Contingência,0.0,BRASIL SEM MISERIA
7,Reserva de Contingência,0.0,BRASIL SEM MISERIA
8,Reserva de Contingência,0.0,VIOLENCIA NO TRANSITO
9,Reserva de Contingência,0.0,VIOLENCIA NO TRANSITO


### 14 - 

In [13]:
df = pd.read_sql_query("SELECT VALOR_DESPESA, NOME_ORGAO FROM (DESPESA_PUBLICA NATURAL JOIN ORGAO) WHERE NOME_ORGAO = 'Ministério da Fazenda'", conn)
df

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


### 15 - 

In [14]:
df = pd.read_sql_query("SELECT NOME_PROGRAMA_GOVERNO, VALOR_DESPESA FROM ((DESPESA_PUBLICA NATURAL JOIN ORGAO) NATURAL JOIN PROGRAMA_GOVERNO) WHERE NOME_ORGAO = 'Ministério da Saúde'", conn)
df

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
5,VIVER SEM LIMITE,0.0
6,VIVER SEM LIMITE,0.0
7,VIVER SEM LIMITE,0.0
8,VIVER SEM LIMITE,0.0
9,VIVER SEM LIMITE,0.0
